<h1>1.8. Lectura de imágenes

In [1]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits

<h1>1.8.1. Trabajando con directorios
    

In [2]:
data_directory = 'path/to/my/images'

<h1>1.8.2. Generar algunas imágenes falsas

Las celdas a continuación generan algunas imágenes falsas para usar más tarde en el cuaderno.

In [3]:
from pathlib import Path
from itertools import cycle

import numpy as np

image_path = Path(data_directory)

image_path.mkdir(parents=True, exist_ok=True)

images_to_generate = {
    'BIAS': 5,
    'DARK': 10,
    'FLAT': 3,
    'LIGHT': 10
}

exposure_times = {
    'BIAS': [0.0],
    'DARK': [5.0, 30.0],
    'FLAT': [5.0, 6.1, 7.3],
    'LIGHT': [30.0],
}

filters = {
    'FLAT': 'V',
    'LIGHT': 'V'
}

objects = {
    'LIGHT': ['m82', 'xx cyg']
}

image_size = [300, 200]

image_number = 0
for image_type, num in images_to_generate.items():
    exposures = cycle(exposure_times[image_type])
    try:
        filts = cycle(filters[image_type])
    except KeyError:
        filts = []
    
    try:
        objs = cycle(objects[image_type])
    except KeyError:
        objs = []
    for _ in range(num):
        img = CCDData(data=np.random.randn(*image_size), unit='adu')
        img.meta['IMAGETYP'] = image_type
        img.meta['EXPOSURE'] = next(exposures)
        if filts:
            img.meta['FILTER'] = next(filts)
        if objs:
            img.meta['OBJECT'] = next(objs)
        image_name = str(image_path / f'img-{image_number:04d}.fits')
        img.write(image_name)
        print(image_name)
        image_number += 1

path\to\my\images\img-0000.fits
path\to\my\images\img-0001.fits
path\to\my\images\img-0002.fits
path\to\my\images\img-0003.fits
path\to\my\images\img-0004.fits
path\to\my\images\img-0005.fits
path\to\my\images\img-0006.fits
path\to\my\images\img-0007.fits
path\to\my\images\img-0008.fits
path\to\my\images\img-0009.fits
path\to\my\images\img-0010.fits
path\to\my\images\img-0011.fits
path\to\my\images\img-0012.fits
path\to\my\images\img-0013.fits
path\to\my\images\img-0014.fits
path\to\my\images\img-0015.fits
path\to\my\images\img-0016.fits
path\to\my\images\img-0017.fits
path\to\my\images\img-0018.fits
path\to\my\images\img-0019.fits
path\to\my\images\img-0020.fits
path\to\my\images\img-0021.fits
path\to\my\images\img-0022.fits
path\to\my\images\img-0023.fits
path\to\my\images\img-0024.fits
path\to\my\images\img-0025.fits
path\to\my\images\img-0026.fits
path\to\my\images\img-0027.fits


<h1>1.8.3. Opción 1: Lectura de una sola imagen con astropy.io.fits

Esta opción le brinda la mayor flexibilidad, pero es la que menos se adapta específicamente a las imágenes CCD. Lo que lee es una lista de extensiones FITS; primero debe seleccionar el que desee y luego acceder a los datos o al encabezado que desee.

In [4]:
image_name = 'img-0001.fits'

image_path = Path(data_directory) / image_name

hdu_list = fits.open(image_path)
hdu_list.info()

Filename: path\to\my\images\img-0001.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       8   (200, 300)   float64   


In [5]:
hdu = hdu_list[0]
hdu.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'BIAS    '                                                            
EXPOSURE=                  0.0                                                  
BUNIT   = 'adu     '                                                            

In [6]:
hdu.data

array([[-0.82652245,  0.20148992, -0.02068813, ...,  0.96122313,
         1.19309966, -0.64795018],
       [-0.71984267,  0.22459072,  2.03059389, ...,  0.47681891,
         1.01641507,  1.44090193],
       [-0.08757494,  1.79636279, -0.27188521, ..., -1.38993545,
        -0.98638123,  0.84790881],
       ...,
       [ 1.24465316, -0.47980438,  1.0045071 , ..., -2.29077912,
         0.11536791,  0.3263036 ],
       [ 0.55463522, -1.02198091,  2.35184838, ..., -0.4216539 ,
        -0.58326207,  0.82877374],
       [ 1.73377048,  1.77274741, -0.93313229, ...,  0.24228633,
        -1.68681993,  1.34694421]])

<h1>1.8.4. Opción 2: Usar CCDDatapara leer en una sola imagen 

stropy contiene un CCDDataobjeto para representar una sola imagen. No es tan flexible como usar astrop.io.fitsdirectamente (por ejemplo, asume que solo hay una extensión FITS y que contiene datos de imagen), pero configura varias propiedades que facilitan el trabajo con los datos.

Leeremos en la misma imagen única que hicimos en el ejemplo anterior, img-0001.fits.

In [7]:
ccd = CCDData.read(image_path)

In [8]:
ccd.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'BIAS    '                                                            
EXPOSURE=                  0.0                                                  
BUNIT   = 'adu     '                                                            

In [9]:
ccd.data

array([[-0.82652245,  0.20148992, -0.02068813, ...,  0.96122313,
         1.19309966, -0.64795018],
       [-0.71984267,  0.22459072,  2.03059389, ...,  0.47681891,
         1.01641507,  1.44090193],
       [-0.08757494,  1.79636279, -0.27188521, ..., -1.38993545,
        -0.98638123,  0.84790881],
       ...,
       [ 1.24465316, -0.47980438,  1.0045071 , ..., -2.29077912,
         0.11536791,  0.3263036 ],
       [ 0.55463522, -1.02198091,  2.35184838, ..., -0.4216539 ,
        -0.58326207,  0.82877374],
       [ 1.73377048,  1.77274741, -0.93313229, ...,  0.24228633,
        -1.68681993,  1.34694421]])

<h1>1.8.5. Opción 3: Trabajar con un directorio de imágenes usando ImageFileCollection

In [10]:
from ccdproc import ImageFileCollection
im_collection = ImageFileCollection(data_directory)

<h1>1.8.5.1. Resumen del contenido del directorio 

In [11]:
im_collection.summary

file,simple,bitpix,naxis,naxis1,naxis2,imagetyp,exposure,bunit,filter,object
str13,bool,int32,int32,int32,int32,str5,float64,str3,object,object
img-0000.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0001.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0002.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0003.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0004.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0005.fits,True,-64,2,200,300,DARK,5.0,adu,--,--
img-0006.fits,True,-64,2,200,300,DARK,30.0,adu,--,--
img-0007.fits,True,-64,2,200,300,DARK,5.0,adu,--,--
img-0008.fits,True,-64,2,200,300,DARK,30.0,adu,--,--


<h1>1.8.5.2. Filtrado e iteración sobre imágenes

In [12]:
for a_flat in im_collection.hdus(imagetyp='FLAT'):
    print(a_flat.header['EXPOSURE'])

5.0
6.1
7.3


In [13]:
for a_flat, fname in im_collection.hdus(imagetyp='LIGHT', object='m82', return_fname=True):
    print(f'In file {fname} the exposure is:', a_flat.header['EXPOSURE'], 'with standard deviation ', a_flat.data.std())

In file img-0018.fits the exposure is: 30.0 with standard deviation  1.0014148044763476
In file img-0020.fits the exposure is: 30.0 with standard deviation  1.0022998102462952
In file img-0022.fits the exposure is: 30.0 with standard deviation  1.0020025455812158
In file img-0024.fits the exposure is: 30.0 with standard deviation  1.0047243089968811
In file img-0026.fits the exposure is: 30.0 with standard deviation  1.0003967215958114


In [14]:
for a_flat, fname in im_collection.ccds(bunit='ADU', return_fname=True):
    print(a_flat.unit)

adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu


In [15]:
a_flat.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'LIGHT   '                                                            
EXPOSURE=                 30.0                                                  
FILTER  = 'V       '                                                            
OBJECT  = 'xx cyg  '                                                            
BUNIT   = 'adu     '                                                            